In [1]:
%env TORCHDYNAMO_VERBOSE=1

env: TORCHDYNAMO_VERBOSE=1


In [2]:
import librosa
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor
from transformers.pipelines.automatic_speech_recognition import AutomaticSpeechRecognitionPipeline

torch.set_float32_matmul_precision("high")

/home/demontego/whisper-on-speed/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

In [4]:
model_id = "openai/whisper-large-v3-turbo"
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True,
).eval().cuda()
processor = AutoProcessor.from_pretrained(model_id)

In [12]:
audio, _ = librosa.load("/home/demontego/whisper/tests/test_data/output.wav", sr=16000)
input_features = processor(audio, sampling_rate=16000, return_tensors="pt").input_features
input_features = input_features.to(torch.float16).cuda()

In [6]:
model.generation_config.cache_implementation = "static"
model.generation_config.max_new_tokens = 324
model.forward = torch.compile(model.forward, mode="reduce-overhead", fullgraph=True)

In [10]:
pipe = AutomaticSpeechRecognitionPipeline(
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch.float16,
    stride_length_s=10/6,
    chunk_length_s=10,
    device=device,
)

Device set to use cuda


In [8]:
generate_kwargs = {
    'max_new_tokens': 444,
    'num_beams': 3,
    'condition_on_prev_tokens': False,
    'compression_ratio_threshold': 1.35,
    'temperature': (0.0, 0.2, 0.4, 0.6, 0.8, 1.0),
    'logprob_threshold': -1.0,
}

In [13]:
result = pipe(audio, generate_kwargs=generate_kwargs, return_timestamps=True)
from pprint import pprint
pprint(result)

/home/demontego/whisper-on-speed/.venv/lib/python3.13/site-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.


{'chunks': [{'text': ' Thank you. Я читаю про дипсик и просто как индекс или '
                     'до этого не додумались просто нам прикольная прям '
                     'архитектуру что типа на поверхность что-то лежало.',
             'timestamp': (0.0, 35.19)},
            {'text': ' Там способ обучения,', 'timestamp': (35.85, 37.47)},
            {'text': ' типа, он', 'timestamp': (37.65, 38.43)},
            {'text': ' много раз обсуждался, но почему-то я редко видел его в '
                     'применении',
             'timestamp': (38.43, 53.67)},
            {'text': ' а я знаю почему они пошли по этому пути.',
             'timestamp': (53.67, 56.21)},
            {'text': ' Они начали обучать давно очень модель кодинга.',
             'timestamp': (56.49, 60.49)},
            {'text': ' DeepSeek Coder у них была лучшая модель',
             'timestamp': (60.71, 63.22)},
            {'text': ' среди всех. Даже лучше, чем сейчас',
             'timestamp': (63.22, 65.98)}